In [1]:
#==深度模型推荐系统 -- NFM==
#一、背景
#FM模型仅局限于线性表达和二阶交互， 无法胜任生活中各种具有复杂结构和规律性的真实数据，针对FM的这点不足， 
#思想是将FM融合进DNN的策略，通过引进了一个特征交叉池化层的结构，使得FM与DNN进行了完美衔接，组合了FM的建模低阶特征交互能力和DNN学习高阶特征交互和非线性的能力，形成了深度学习时代的神经FM模型(NFM)。
#2017 新加坡国立大学 何向南等 在SIGIR会议 提出。
#公式改变：对比FM，变化第三项（公式里的二阶交叉项），用一个表达能力更强的函数来替代原FM中二阶隐向量内积的部分。实现线性二阶到更复杂灵活的高阶变化。
#//变幻和突破从公式着手。//看公式是有用的……大师都是从这里开始改进发明新模型的。
#进一步的，把这个函数用神经网络来代替，神经网络理论上可以拟合任何复杂能力的函数。
#这个加的神经网络先做了底层的交叉，顶层接了DNN。最终形成了 --> NFM网络
#神经网络底层的Bi-Interaction Pooling 交叉池化层是 NFM的核心结构。

#二、模型结构
# 2.1 Input和Embedding层
# 稀疏离散特征居多，这两层还是一般套路：先one-hot, 然后会通过embedding，处理成稠密低维的。
# 2.2 Bi-Interaction Pooling layer
# 特征交叉池化层是本网络的核心创新，正是因为这个结构，实现了FM与DNN的无缝连接， 组成了一个大的网络，且能够正常的反向传播。
# 做两个向量的元素积操作，即两个向量对应维度相乘得到的元素积向量（不是点乘）。
# 之前的FM到这里其实就完事了， 上面就是输出了，而这里很大的一点改进就是加入特征池化层之后， 把二阶交互的信息合并， 且上面接了一个DNN网络， 这样就能够增强FM的表达能力了， 因为FM只能到二阶， 而这里的DNN可以进行多阶且非线性，只要FM把二阶的学习好了， DNN这块学习来会更加容易
#【对比与FM的二阶交叉：原理：NFM中的特征交叉是两个向量的元素积操作，即两个向量对应维度相乘得到的元素积向量（不是点乘）。
#也就是这一个交叉完了之后k个维度不求和，最后会得到一个k维向量。送到后面去做池化和组合，再送入新模型（DNN）继续学习。
#FM的特征交叉是做两个隐向量的内积最后得到一个数，就输出了。】
# 如果不加DNN， NFM就退化成了FM，所以改进的关键就在于加了一个这样的层，组合了一下二阶交叉的信息，然后又给了DNN进行高阶交叉的学习，成了一种“加强版”的FM。
# Bi-Interaction层不需要额外的模型学习参数，更重要的是它在一个线性的时间内完成计算，和FM一致的，时间复杂度一样。
# 2.3 隐藏层
# 就是一个全连接的神经网络， DNN在进行特征的高层非线性交互上有着天然的学习优势。
# 2.4 预测层
# 最后一层的结果直接过一个隐藏层，但注意由于这里是回归问题，没有加sigmoid激活。
# NFM相比较于其他模型的核心创新点是特征交叉池化层，基于它，实现了FM和DNN的无缝连接，使得DNN可以在底层就学习到包含更多信息的组合特征，这时候，就会减少DNN的很多负担，只需要很少的隐藏层就可以学习到高阶特征信息。NFM相比之前的DNN， 模型结构更浅，更简单，但是性能更好，训练和调参更容易。集合FM二阶交叉线性和DNN高阶交叉非线性的优势，非常适合处理稀疏数据的场景任务。在对NFM的真实训练过程中，也会用到像Dropout和BatchNormalization这样的技术来缓解过拟合和在过大的改变数据分布。
# //总结下来感觉就是简单实用。

# 其它与FM结合的模型：
# DeepFM 是 Wide&Deep 框架改进，将其中 wide 部分换成 fm 进行二阶交叉。
#都是可以随意组合的

#FM：Factorization Machine 因子分解机，引入机器学习算法模型来做MF（矩阵分解 如 商品与用户的隐藏关系矩阵），将手动特征交叉升级为二阶交叉项公式。

In [2]:
# 三、代码实现
def NFM(linear_feature_columns, dnn_feature_columns):
    """
    搭建NFM模型，上面已经把所有组块都写好了，这里拼起来就好
    :param linear_feature_columns: A list. 里面的每个元素是namedtuple(元组的一种扩展类型，同时支持序号和属性名访问组件)类型，表示的是linear数据的特征封装版
    :param dnn_feature_columns: A list. 里面的每个元素是namedtuple(元组的一种扩展类型，同时支持序号和属性名访问组件)类型，表示的是DNN数据的特征封装版
    """
    # 构建输入层，即所有特征对应的Input()层， 这里使用字典的形式返回， 方便后续构建模型
    # 构建模型的输入层，模型的输入层不能是字典的形式，应该将字典的形式转换成列表的形式
    # 注意：这里实际的输入与Input()层的对应，是通过模型输入时候的字典数据的key与对应name的Input层
    dense_input_dict, sparse_input_dict = build_input_layers(linear_feature_columns+dnn_feature_columns)
    input_layers = list(dense_input_dict.values()) + list(sparse_input_dict.values())
    
    # 线性部分的计算 w1x1 + w2x2 + ..wnxn + b部分，dense特征和sparse两部分的计算结果组成，具体看上面细节
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, linear_feature_columns)
    
    # DNN部分的计算
    # 首先，在这里构建DNN部分的embedding层，之所以写在这里，是为了灵活的迁移到其他网络上，这里用字典的形式返回
    # embedding层用于构建FM交叉部分以及DNN的输入部分
    embedding_layers = build_embedding_layers(dnn_feature_columns, sparse_input_dict, is_linear=False)
    
    # 过特征交叉池化层
    pooling_output = get_bi_interaction_pooling_output(sparse_input_dict, dnn_feature_columns, embedding_layers)
    
    # 加个BatchNormalization
    pooling_output = BatchNormalization()(pooling_output)
    
    # dnn部分的计算
    dnn_logits = get_dnn_logits(pooling_output)
    
    # 线性部分和dnn部分的结果相加，最后再过个sigmoid
    output_logits = Add()([linear_logits, dnn_logits])
    output_layers = Activation("sigmoid")(output_logits)
    
    model = Model(inputs=input_layers, outputs=output_layers)
    
    return model

In [ ]:
# 四、思考
# NFM中的特征交叉与FM中的特征交叉有何异同，分别从原理和代码实现上进行对比分析
#原理：NFM中的特征交叉是两个向量的元素积操作，即两个向量对应维度相乘得到的元素积向量（不是点乘）。
#也就是这一个交叉完了之后k个维度不求和，最后会得到一个k维向量。送到后面去做池化和组合，再送入新模型（DNN）继续学习。
#FM的特征交叉是做两个隐向量的内积最后得到一个数，就输出了。
#代码上：
#NFM:
# embedding_layers = build_embedding_layers(dnn_feature_columns, sparse_input_dict, is_linear=False)
#         # 过特征交叉池化层
# pooling_output = get_bi_interaction_pooling_output(sparse_input_dict, dnn_feature_columns, embedding_layers)

#FM:
#调包实现参考：
# fm = pylibfm.FM(num_factors=50, num_iter=10, verbose=True, task="classification", 
# initial_learning_rate=0.0001, learning_rate_schedule="optimal")
# fm.fit(X_train,y_train)